## Regularization
#### Example using Regularized Adjusted Plus Minus from Basketball

Source: http://nbviewer.jupyter.org/gist/EvanZ/48bf713ce9eb14f28d58

In [1]:
import pandas as pd
import json
from pprint import pprint
from sklearn.feature_extraction import DictVectorizer
from sklearn import linear_model
import statsmodels.api as sm
import numpy as np
from IPython.display import Image

/Users/tpetersen/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### What is Plus/Minus
In basketball, there are many valuable plays that aren't tracked in the box score
* Taking a charge
* Deflecting a pass
* Boxing out to allow a teammate to get a rebound
* Hockey assists
* Setting good screens

To account for this, basketball analysts began to measure Plus/Minus in order to measure how well the team performs when the player is on the court.

* A player's plus/minus is the scoring margin between their team and the other team while they are on the court

In [2]:
Image(url='https://winninghoops.com/wp-content/uploads/2017/03/StottsChart_A.png')

### Issues with Plus/Minus
* Mediocre players on good teams benefit from the great lineups they play on
* Very difficult for a great player on bad team to have a positive plus/minus

### Adjusted Plus/Minus
* An attempt to control for strength of temmates and opponents while players are on the court
* Dummy variables used for each player while they are on the court
* Regression analysis estimates the impact (or coefficients) of each player

### How to Calculate Adjusted Plus/Minus

#### First we'll start with Stint Data
* Stint or shift data evaluates the periods in games without substitution
* Each 10-man unit shift represents a single observation

In [3]:
data = []  # create an empty array to hold all the stints
# open the file and read the data!
with open('matchups-2015-final.json') as units_file:
    for j in units_file:
        data.append(json.loads(j))

In [4]:
pprint(data[0])

{'Lakers': {'entered': ['Robert Sacre'],
            'exited': ['Jordan Hill'],
            'on': ['Ronnie Price',
                   'Wayne Ellington',
                   'Wesley Johnson',
                   'Carlos Boozer',
                   'Robert Sacre'],
            'stats': {'dreb': 1,
                      'drebx': 0,
                      'fg2m': 1,
                      'fg3m': 3,
                      'fgm': 4,
                      'fgx': 0,
                      'foul': 0,
                      'fta': 0,
                      'ftm': 0,
                      'non_steal_tov': 0,
                      'oreb': 0,
                      'orebx': 0,
                      'poss': 4,
                      'pts': 11,
                      'team_tov': 0,
                      'time': 0,
                      'tov': 0}},
 'Warriors': {'entered': ['Andre Iguodala'],
              'exited': ['Marreese Speights'],
              'on': ['Stephen Curry',
                     'Klay Thompson

#### Next we will evaluate the net points per possession
* Net points will be divided by number of possessions
* Home team players will be indicated with a 1 and away team with a -1
* Weights will be used as the number of possessions

In [5]:
units = []
points = []
weights = []

for d in data:
    home = d['home']
    away = d['away']
    home_poss = d[home]['stats']['poss']
    away_poss = d[away]['stats']['poss']

    stint = {name: 1 for name in d[home]['on']}
    stint.update({name: -1 for name in d[away]['on']})

    if (home_poss+away_poss) >= 1:  # to avoid some ill-conditioning we only use stints that have possessions >= 1
        ortg = 100 * (d[home]['stats']['pts']-d[away]['stats']['pts']) / (home_poss+away_poss)
        units.append(stint)
        points.append(ortg)
        weights.append(home_poss+away_poss)

u = DictVectorizer(sparse=False)
u_mat = u.fit_transform(units)

In [6]:
player_matrix = pd.DataFrame(u_mat, columns=u.get_feature_names())
points_per_100 = pd.Series(points)
num_possession = pd.Series(weights)

In [7]:
player_matrix.head()

,A.J. Price,Aaron Brooks,Aaron Gordon,Adreian Payne,Al Horford,Al Jefferson,Al-Farouq Aminu,Alan Anderson,Alec Burks,Alex Kirk,...,Will Barton,Will Bynum,Will Cherry,Willie Green,Wilson Chandler,Xavier Henry,Zach LaVine,Zach Randolph,Zaza Pachulia,Zoran Dragic
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
points_per_100.head()

0     87.5
1      0.0
2    -40.0
3    120.0
4    100.0
dtype: float64

In [9]:
players = pd.read_csv('NBA_2015_totals.csv')
transp_pmat = player_matrix.T.reset_index()
transp_p = transp_pmat.merge(players[['Player','MP']], left_on='index', right_on='Player', how='left').drop('index', axis=1)
transp_p.loc[transp_p['MP']<1200, 'Player'] = 'BASELINE'
transp_p = transp_p[transp_p['Player']!='BASELINE'].dropna(subset=['Player']).reset_index(drop=True)
player_matrix_thresh = transp_p.set_index('Player').drop('MP', axis=1).T

In [10]:
player_matrix_thresh.shape

(38421, 240)

In [11]:
player_matrix_thresh.head()

Player,Aaron Brooks,Al Horford,Al Jefferson,Al-Farouq Aminu,Alan Anderson,Alex Len,Amar'e Stoudemire,Amir Johnson,Andre Drummond,Andre Iguodala,...,Tyreke Evans,Tyson Chandler,Victor Oladipo,Wayne Ellington,Wesley Johnson,Wesley Matthews,Wilson Chandler,Zach LaVine,Zach Randolph,Zaza Pachulia
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [12]:
apm_model = sm.WLS(points_per_100, player_matrix_thresh, weights=num_possession).fit()

In [13]:
apm_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.018
Model:                            WLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     2.921
Date:                Mon, 19 Feb 2018   Prob (F-statistic):           4.17e-46
Time:                        19:57:37   Log-Likelihood:            -2.0949e+05
No. Observations:               38421   AIC:                         4.195e+05
Df Residuals:                   38181   BIC:                         4.215e+05
Df Model:                         240                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Aaron Brooks                 3.7319      2.399      1.556      0.120      -0.970       8.434
Al Horford                  -0.0639      2.187     -0.029      0.977      -4.351       4.223
Al Jefferson                 1.2283      2.815      0.436      0.663      -4.289       6.745
Al-Farouq Aminu              2.8225      2.206      1.279      0.201      -1.502       7.147
Alan Anderson                3.2079      1.869      1.716      0.086      -0.456       6.871
Alex Len                    -0.5675      2.046     -0.277      0.781      -4.577       3.442
Amar'e Stoudemire            0.7792      2.047      0.381      0.703      -3.233       4.791
Amir Johnson                 0.9672      2.332      0.415      0.678      -3.604       5.539
Andre Drummond               2.3037      2.105      1.094      0.274      -1.823       6.430
Andre Iguodala               1.5769      1.769      0.891      0.373      -1.890       5.044
Andre Miller                 2.5700      2.385      1.078      0.281      -2.104       7.244
Andre Roberson               3.4912      2.364      1.477      0.140      -1.143       8.126
Andrew Bogut                 1.0194      2.092      0.487      0.626      -3.080       5.119
Andrew Wiggins               3.2709      1.930      1.695      0.090      -0.512       7.054
Anthony Davis                7.4099      2.193      3.379      0.001       3.112      11.708
Anthony Morrow               5.4606      1.836      2.974      0.003       1.862       9.059
Anthony Tolliver             4.9115      2.106      2.332      0.020       0.783       9.040
Arron Afflalo                1.9141      1.758      1.089      0.276      -1.531       5.360
Austin Rivers               -2.0721      1.712     -1.210      0.226      -5.428       1.283
Avery Bradley                2.4797      2.077      1.194      0.233      -1.592       6.551
Ben McLemore                 4.3235      2.103      2.056      0.040       0.202       8.445
Beno Udrih                  -1.5178      2.501     -0.607      0.544      -6.421       3.385
Bismack Biyombo              1.0846      2.793      0.388      0.698      -4.390       6.559
Blake Griffin                1.0165      2.348      0.433      0.665      -3.586       5.619
Bojan Bogdanovic            -0.0199      1.799     -0.011      0.991      -3.546       3.506
Boris Diaw                   1.1090      1.895      0.585      0.558      -2.605       4.823
Bradley Beal                 4.0244      1.938      2.077      0.038       0.226       7.823
Brandan Wright              -1.1369      1.907     -0.596      0.551      -4.874       2.600
Brandon Bass                -1.0841      2.100     -0.516      0.606      -5.201       3.032
Brandon Knight              -0.4782      1.828     -0.262      0.794      -4.061       3.104
Brian Roberts 

In [14]:
apm_df = pd.concat([apm_model.params.T,
                   np.round(apm_model.pvalues.T,2)], axis=1).reset_index().sort_values(0, ascending=False).reset_index(drop=True)

In [15]:
apm_df.columns = ['Player','APM','p_value']

In [16]:
apm_df.head(10)

,Player,APM,p_value
0,Kyle Korver,8.430022,0.00
1,James Harden,8.291537,0.00
2,Kawhi Leonard,8.161164,0.00
3,Stephen Curry,8.137417,0.00
4,Derrick Rose,7.893078,0.00
5,Khris Middleton,7.417472,0.00
6,Anthony Davis,7.409907,0.00
7,LeBron James,7.119602,0.00
8,Chris Paul,6.992629,0.01
9,Zach Randolph,6.905481,0.00


In [17]:
apm_df.tail()

,Player,APM,p_value
235,Derrick Williams,-3.487545,0.08
236,Robin Lopez,-3.558054,0.13
237,Tony Parker,-3.582240,0.16
238,Zach LaVine,-4.255777,0.03
239,Chris Kaman,-5.817944,0.02


### Issue with Adjusted Plus/Minus
Multicollinearity: Certain players may always play in the same lineups together, or they always substitute for each other
![gortat_howard](https://nbcprobasketballtalk.files.wordpress.com/2011/03/dwight-howard-marcin-gortat1.jpg?w=320&h=201&crop=1)

### How to solve for multicollinearity - Ridge Regression
* When multicollinearity occurs, least squares estimates are unbiased, but their variances are large so they may be far from the true value. 
* By adding a degree of bias to the regression estimates, ridge regression reduces the standard errors.
* Reduces the variance of the estimates.
* While biased, the reduced variance of ridge estimates often results in a smaller mean square error when compared to least-squares estimates.

### Why is it called Ridge
* If there's multicollinearity, there is a "ridge" in the likelihood function (likelihood is a function of the $\beta$'s). This in turn yields a long "valley" in the $RSS$ (since $RSS=−2log$).

* Ridge regression "fixes" the ridge - it adds a penalty that turns the ridge into a nice peak in likelihood space, equivalently a nice depression in the criterion we're minimizing:

![Alt Text](http://i.stack.imgur.com/1d8XV.png)

* In 1959 A.E. Hoerl [1] introduced ridge analysis for response surface methodology, and it very soon [2] became adapted to dealing with multicollinearity in regression ('ridge regression'). See for example, the discussion by R.W. Hoerl in [3], where it describes Hoerl's (A.E. not R.W.) use of contour plots of the response surface* in the identification of where to head to find local optima (where one 'heads up the ridge'). In ill-conditioned problems, the issue of a very long ridge arises, and insights and methodology from ridge analysis are adapted to the related issue with the likelihood/RSS in regression, producing ridge regression.

[1]: Hoerl, A.E. (1959). Optimum solution of many variables equations. Chemical Engineering Progress, 55 (11) 69-78.

[2]: Hoerl, A.E. (1962). Applications of ridge analysis to regression problems. Chemical Engineering Progress, 58 (3) 54-59.

[3] Hoerl, R.W. (1985). Ridge Analysis 25 Years Later. American Statistician, 39 (3), 186-192 

[Source](http://stats.stackexchange.com/questions/151304/why-is-ridge-regression-called-ridge-why-is-it-needed-and-what-happens-when)

## How is it performed?

* Ordinary least squares seeks to minimize the sum of squared residuals, which can be compactly written as 

$argmin \atop{\widehat\beta \in {\rm I\!R}^p}$ $||y- \mathbf{X}\beta||_2^2$


* The inclusion of an $L_2$ penalty shrinks the estimated coefficients towards zero


* Given a response vector $y \in {\rm I\!R}^n$ and a predictor matrix $\mathbf{X}\in{\rm I\!R}^{nxp}$ the ridge coefficients are defined as 

$argmin \atop{\widehat\beta \in {\rm I\!R}^p}$ $||y- \mathbf{X}\beta||_2^2$ + $\lambda||\beta||_2^2$

* Where  $||y- \mathbf{X}\beta||_2^2$ is the loss


* And  $\lambda||\beta||_2^2$ is the penalty


* Here $\lambda≥0$ is a **tuning parameter** controlling the strength of the penalty term
* When $\lambda=0$ the estimates are the same as the linear regression
* When $\lambda = \infty$ then the coefficients equal zero ($\widehat\beta^{ridge}=0$)

$argmin \atop{\widehat\beta \in {\rm I\!R}^p}$ $||y- \mathbf{X}\beta||_2^2$

$s$ $=$ $\hat{r}$  $/$  $\bar{r}$

## Ridge Regression

* $L_2$ regularization is also known as Ridge Regression

* Shrinks the coeﬃcients by imposing a penalty on their size. 

* Ridge coeﬃcients minimize a penalized residual sum of squares. 

## $L_2$ vs $L_1$ Penalty

The Ridge Regression penalty is known as the $L_2$ penalty:

$||\beta||^2$

The $L_1$ penalty is used in LASSO (Least Absolute Shrinkage and Selection Operator):

$||\beta||_1$

* The general idea is that you are restricting the "space" in which your coefficients can be fit. This means you are shriking the coefficient space

* Due to the shape or space for the regression problem and the shape of our penalty box, many of the "optimal" coefficients will be close to zero for Ridge Regression and exactly zero for LASSO Regression.


## Difference ?  Tradeoffs?

** Ridge **
* Positives
    * Introduces some bias, but can greatly reduce the variance,
resulting in a better mean-squared error
    *  Performs particularly well when there is a subset of true coeffcients that are small or even zero


* Negatives
    * Doesn't do as well when all of the true coeffcients are moderately large
    * Cannot perform variable selection

    

** Lasso **
* Positives
    * Feature Selection

* Negatives
    * For "large p, small n" case (high-dimensional data with few examples), the LASSO selects at most n variables before it saturates
    * If there is a group of highly correlated variables, then the LASSO tends to select one variable from a group and ignore the others ([Quora on $L_1$ vs. $L_2$ Regularization](https://www.quora.com/What-is-the-difference-between-L1-and-L2-regularization))
    
** Elastic Net **
* Overcomes the limitations of LASSO by adding the quadratic part to the penalty, which is used in ridge regression 
    * [Wiki](https://en.wikipedia.org/wiki/Elastic_net_regularization)

### Bias vs. Variance
<img src="https://qph.ec.quoracdn.net/main-qimg-01c15f01cf6a56c19313c2791d5a9ae1-c" style="width: 550px;"/>

In [18]:
Image(url='http://scott.fortmann-roe.com/docs/docs/BiasVariance/biasvariance.png')

### Regularized Adjusted Plus/Minus
* Coefficients can be thought of "amount of differential contribution with respect to variation"
* Cross validation used to identify the proper lambda 

In [19]:
clf = linear_model.RidgeCV(alphas=(np.array([0.01, 0.1, 1.0, 10, 100, 500, 750, 1000, 1500, 2000, 5000])), cv=5)
clf.fit(player_matrix_thresh, points_per_100, sample_weight=num_possession)

RidgeCV(alphas=array([  1.00000e-02,   1.00000e-01,   1.00000e+00,   1.00000e+01,
         1.00000e+02,   5.00000e+02,   7.50000e+02,   1.00000e+03,
         1.50000e+03,   2.00000e+03,   5.00000e+03]),
    cv=5, fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

In [20]:
clf.alpha_

1500.0

In [21]:
ratings = []
n = -1
for player in player_matrix_thresh.columns:
    n+=1
    ratings.append((player, clf.coef_[n]))
ratings.sort(key=lambda tup: tup[1], reverse=True)
rapm_frame = pd.DataFrame(ratings, columns=['Player', 'RAPM'])
rapm_v_apm = rapm_frame.merge(apm_df, on='Player')
rapm_v_apm['APM_rank'] = rapm_v_apm['APM'].rank(ascending=False)
rapm_v_apm['RAPM_rank'] = rapm_v_apm['RAPM'].rank(ascending=False)
rapm_v_apm['RAPM_change'] = rapm_v_apm['APM_rank'] - rapm_v_apm['RAPM_rank']

In [22]:
rapm_v_apm.head(10)

,Player,RAPM,APM,p_value,APM_rank,RAPM_rank,RAPM_change
0,Kyle Korver,5.424223,8.430022,0.00,1.0,1.0,0.0
1,Stephen Curry,5.200709,8.137417,0.00,4.0,2.0,2.0
2,Anthony Davis,5.136864,7.409907,0.00,7.0,3.0,4.0
3,Kawhi Leonard,5.054951,8.161164,0.00,3.0,4.0,-1.0
4,Draymond Green,5.009834,6.658933,0.00,13.0,5.0,8.0
5,James Harden,4.813614,8.291537,0.00,2.0,6.0,-4.0
6,LeBron James,4.793611,7.119602,0.00,8.0,7.0,1.0
7,Khris Middleton,4.570415,7.417472,0.00,6.0,8.0,-2.0
8,Chris Paul,4.402360,6.992629,0.01,9.0,9.0,0.0
9,Zach Randolph,4.273069,6.905481,0.00,10.0,10.0,0.0


In [23]:
rapm_v_apm.sort_values('RAPM_change').head(10)

,Player,RAPM,APM,p_value,APM_rank,RAPM_rank,RAPM_change
181,Kobe Bryant,-0.587837,1.516155,0.58,117.0,182.0,-65.0
124,Andrew Wiggins,0.442423,3.270931,0.09,69.0,125.0,-56.0
141,Marvin Williams,0.234664,2.589297,0.30,87.0,142.0,-55.0
159,Ryan Kelly,-0.076078,1.795695,0.44,109.0,160.0,-51.0
202,Solomon Hill,-1.083272,0.398300,0.86,158.0,203.0,-45.0
192,Hollis Thompson,-0.890355,0.620899,0.73,149.0,193.0,-44.0
149,Jamal Crawford,0.062984,1.720762,0.35,112.0,150.0,-38.0
204,Kirk Hinrich,-1.110395,0.025941,0.99,167.0,205.0,-38.0
163,Mario Chalmers,-0.235961,1.140078,0.51,127.0,164.0,-37.0
121,Andre Miller,0.486790,2.569966,0.28,88.0,122.0,-34.0


In [24]:
rapm_v_apm.sort_values('RAPM_change', ascending=False).head(10)

,Player,RAPM,APM,p_value,APM_rank,RAPM_rank,RAPM_change
130,Pau Gasol,0.332886,-1.349399,0.59,211.0,131.0,80.0
113,Patrick Patterson,0.740371,-0.034094,0.99,169.0,114.0,55.0
142,David West,0.231271,-0.569701,0.87,194.0,143.0,51.0
69,DeAndre Jordan,1.849053,1.465290,0.57,120.0,70.0,50.0
111,Nene Hilario,0.760752,0.346067,0.88,160.0,112.0,48.0
87,Blake Griffin,1.370197,1.016550,0.67,134.0,88.0,46.0
182,Joakim Noah,-0.597294,-2.181783,0.33,227.0,183.0,44.0
154,Jason Terry,-0.030659,-0.579400,0.76,195.0,155.0,40.0
20,Tony Allen,3.361993,3.965501,0.05,58.0,21.0,37.0
39,Zaza Pachulia,2.805996,3.156171,0.18,75.0,40.0,35.0


## Next Steps
* Jeremias Engelmann has a variant called xRAPM that uses a box score based metric as a prior
* Engelmann was hired by ESPN and xRAPM was used as the basis for Real Plus-Minus
* Player Tracking data will increasingly be used in player evaluation

## Links

[Measuring How NBA Players Help Their Teams Win](http://www.82games.com/comm30.htm)

[Improved NBA Adjusted +/- Using Regularization and Out-of-Sample Testing](http://www.sloansportsconference.com/wp-content/uploads/2015/09/joeSillSloanSportsPaperWithLogo.pdf)

[Deep Dive on Regularized Adjusted Plus Minus I](https://squared2020.com/2017/09/18/deep-dive-on-regularized-adjusted-plus-minus-i-introductory-example/)

[Deep Dive on Regularized Adjusted Plus Minus II](https://squared2020.com/2017/09/18/deep-dive-on-regularized-adjusted-plus-minus-ii-basic-application-to-2017-nba-data-with-r/)

[Solutions for Multicollinearity in Regression](https://www.r-bloggers.com/solutions-for-multicollinearity-in-regression1/)

[Understanding the Bias-Variance Tradeoff](http://scott.fortmann-roe.com/docs/BiasVariance.html)

[How to Calculate RAPM](https://web.archive.org/web/20140717131517/http://www.hickory-high.com/how-to-calculate-rapm/)

[Jeremias Engelmann APBR xRAPM Discussion](http://www.apbr.org/metrics/viewtopic.php?f=2&t=8025&p=13810&hilit=box#p13830)

[A Review of Adjusted Plus/Minus and Stabilization](http://godismyjudgeok.com/DStats/2011/nba-stats/a-review-of-adjusted-plusminus-and-stabilization/)

[Introducing Real Plus-Minus](http://www.espn.com/nba/story/_/id/10740818/introducing-real-plus-minus)